
        
 #In this Notebook:
I am trying different formats of disk storage in python like .csv, .pickle, parquet, .feather etc and comparing the time it takes to read/write 1 million rows of a dataframe and it maintains the datatype after writting and reading back.
 
#Conslusion
We see that for a 4.5 million rows  dataframe parquet format takes 1/20th time (21.1 sec for .csv and 1.7 sec for parquet). Parquet format also take around 1/6th disk space.







In [130]:
import pickle


In [131]:
def storegkData():
    gautam = {'key':'gautam','name':'gautam kumar','age':21,'pay':23000}
    alex = {'key':'alex','name':'alex stuttgart','age':53,'pay':52000}
    
    dt = {}
    dt['gautam'] = gautam
    dt['alex'] = alex
    
    dbfile = open('examPickle','ab')
    
    pickle.dump(dt,dbfile)

    dbfile.close()

In [135]:
def loadgkData():
    dtfile2 = open('examPickle','rb')

    dt = pickle.load(dtfile2)

    for keys in dt:
        print("Keys =>>", keys, dt[keys])
    dtfile2.close()

In [136]:
storegkData()

In [137]:
loadgkData()

Keys =>> omkar {'key': 'Omkar', 'name': 'Omkar Pathak', 'age': 21, 'pay': 40000}
Keys =>> jagdish {'key': 'Jagdish', 'name': 'Jagdish Pathak', 'age': 50, 'pay': 50000}


In [112]:
!pip install pyarrow
!pip install fastparquet

     |████████████████████████████████| 465 kB 4.7 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 2.1 MB 42.5 MB/s eta 0:00:01
     |████████████████████████████████| 11.7 MB 347 kB/s eta 0:00:01
     |████████████████████████████████| 229 kB 50.9 MB/s eta 0:00:01
     |████████████████████████████████| 346 kB 50.3 MB/s eta 0:00:01
  Created wheel for fastparquet: filename=fastparquet-2024.2.0-cp38-cp38-macosx_10_9_x86_64.whl size=698471 sha256=9a98470739776a86253fa6bc3dde0cd77a3014780bd0d4e05435eea04af2596b
  Stored in directory: /Users/user/Library/Caches/pip/wheels/b7/cd/fe/e9f613eb7fb2eb40d3ca9f5893085cbbe2035b58261b1b3172
Successfully built fastparquet
  Attempting uninstall: python-dateutil
    Found existing installation: python-dateutil 2.8.1
    Uninstalling python-dateutil-2.8.1:
      Successfully u

In [48]:
import pandas as pd
import numpy as np

/Users/user/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [49]:


def get_gkdataset(size):
    # Create simple Dataset
    x_df = pd.DataFrame()
    x_df['size'] = np.random.choice(['big','medium','small'], size)
    x_df['age'] = np.random.randint(1, 50, size)
    x_df['team'] = np.random.choice(['red','blue','yellow','green'], size)
    x_df['win'] = np.random.choice(['yes','no'], size)
    dates = pd.date_range('2020-01-01', '2022-12-31')
    x_df['date'] = np.random.choice(dates, size)
    x_df['prob'] = np.random.uniform(0, 1, size)
    return x_df

def set_gkdtypes(x_df):
    # Set data types.
    x_df['size'] = x_df['size'].astype('category')
    x_df['team'] = x_df['team'].astype('category')
    x_df['age'] = x_df['age'].astype('int16')
    x_df['win'] = x_df['win'].map({'yes':True, 'no': False})
    x_df['prob'] = x_df['prob'].astype('float32')
    return x_df

In [51]:
df = get_gkdataset(100000)

In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   size    100000 non-null  object        
 1   age     100000 non-null  int64         
 2   team    100000 non-null  object        
 3   win     100000 non-null  object        
 4   date    100000 non-null  datetime64[ns]
 5   prob    100000 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 4.6+ MB


In [54]:
df.head()

,size,age,team,win,date,prob
0,big,13,green,no,2020-02-02,0.132523
1,big,10,red,yes,2020-08-09,0.550079
2,big,44,blue,yes,2022-08-20,0.978710
3,big,4,green,yes,2022-03-09,0.654751
4,medium,37,yellow,no,2020-06-05,0.231969


In [67]:
%%timeit
df = get_gkdataset(1000000)

810 ms ± 36.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [68]:
%%timeit
df.to_csv("test_csv.csv",index = False)

3.25 s ± 78.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [69]:
!ls -GFlash test_csv.csv

94216 -rw-r--r--  1 user  staff    45M Oct 15 14:44 test_csv.csv


In [70]:
%%timeit
df = pd.read_csv("test_csv.csv")

609 ms ± 11.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [71]:
df.head()

,size,age,team,win,date,prob
0,medium,4,blue,yes,2022-09-27,0.003669
1,small,44,red,yes,2020-12-11,0.836712
2,medium,1,blue,yes,2021-01-28,0.849984
3,small,18,blue,yes,2022-12-20,0.008492
4,medium,3,red,no,2021-06-14,0.922291


In [73]:
## Set datatypes on the columns
gkdf = get_gkdataset(1000000)
set_gkdtypes(gkdf)

,size,age,team,win,date,prob
0,medium,43,green,True,2022-12-14,0.353590
1,medium,31,green,False,2021-07-25,0.007596
2,big,25,green,False,2022-07-20,0.456653
3,big,20,red,False,2020-03-16,0.902980
4,small,6,red,True,2022-03-18,0.391114
...,...,...,...,...,...,...
999995,big,47,green,False,2022-07-11,0.039985
999996,small,29,yellow,True,2022-07-13,0.404151
999997,small,44,green,False,2021-05-15,0.001012
999998,medium,46,yellow,False,2020-11-17,0.761888


In [74]:
gkdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype         
---  ------  --------------    -----         
 0   size    1000000 non-null  category      
 1   age     1000000 non-null  int16         
 2   team    1000000 non-null  category      
 3   win     1000000 non-null  bool          
 4   date    1000000 non-null  datetime64[ns]
 5   prob    1000000 non-null  float32       
dtypes: bool(1), category(2), datetime64[ns](1), float32(1), int16(1)
memory usage: 16.2 MB


In [75]:
gkdf.to_csv("gktest_csv.csv",index = False)


In [78]:
gkdf = pd.read_csv("gktest_csv.csv")
gkdf.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   size    1000000 non-null  object 
 1   age     1000000 non-null  int64  
 2   team    1000000 non-null  object 
 3   win     1000000 non-null  bool   
 4   date    1000000 non-null  object 
 5   prob    1000000 non-null  float64
dtypes: bool(1), float64(1), int64(1), object(3)
memory usage: 39.1+ MB


In [90]:
gkdf = get_gkdataset(1000000)
set_gkdtypes(df)

,size,age,team,win,date,prob
0,big,39,red,False,2022-09-19,0.773951
1,big,23,yellow,False,2020-03-09,0.733663
2,small,24,red,True,2022-01-13,0.346818
3,big,47,red,False,2020-05-23,0.270247
4,big,22,yellow,True,2020-07-17,0.382008
...,...,...,...,...,...,...
999995,big,28,green,True,2020-04-19,0.547172
999996,big,33,red,False,2020-03-12,0.926601
999997,small,13,yellow,True,2021-04-15,0.747553
999998,small,20,blue,True,2020-12-02,0.847842


In [104]:

%timeit df.to_pickle("test.pickle")



In [105]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype         
---  ------  --------------    -----         
 0   size    1000000 non-null  category      
 1   age     1000000 non-null  int16         
 2   team    1000000 non-null  category      
 3   win     1000000 non-null  bool          
 4   date    1000000 non-null  datetime64[ns]
 5   prob    1000000 non-null  float32       
dtypes: bool(1), category(2), datetime64[ns](1), float32(1), int16(1)
memory usage: 16.2 MB


In [109]:
df_pickle = pd.read_pickle("test.pickle")
df_pickle.head()

,size,age,team,win,date,prob
0,big,39,red,False,2022-09-19,0.773951
1,big,23,yellow,False,2020-03-09,0.733663
2,small,24,red,True,2022-01-13,0.346818
3,big,47,red,False,2020-05-23,0.270247
4,big,22,yellow,True,2020-07-17,0.382008


In [111]:
df_pickle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype         
---  ------  --------------    -----         
 0   size    1000000 non-null  category      
 1   age     1000000 non-null  int16         
 2   team    1000000 non-null  category      
 3   win     1000000 non-null  bool          
 4   date    1000000 non-null  datetime64[ns]
 5   prob    1000000 non-null  float32       
dtypes: bool(1), category(2), datetime64[ns](1), float32(1), int16(1)
memory usage: 16.2 MB


Parquet

In [113]:
df = get_gkdataset(1000000)
%timeit df.to_parquet("test.parquet")
%timeit df_parquet = pd.read_parquet("test.parquet")

338 ms ± 18.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
115 ms ± 6.94 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [114]:
!ls -GFlash test.parquet

22936 -rw-r--r--  1 user  staff    11M Oct 15 15:05 test.parquet


In [115]:
df_parquet_columns = pd.read_parquet("test.parquet",columns = ['size','age'])

In [116]:
df_parquet_columns.head()

,size,age
0,small,10
1,big,26
2,medium,12
3,medium,29
4,small,13


#Feather

In [118]:
gkdf = get_gkdataset(1000000)
%timeit gkdf.to_feather("test.feather")
%timeit gkdf_feather = pd.read_feather("test.feather")

187 ms ± 3.92 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
107 ms ± 16.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [119]:
!ls -GFlash test.feather

59688 -rw-r--r--  1 user  staff    29M Oct 15 15:10 test.feather


# Compare All the formats

In [121]:
# CCSV File.
gk_df = get_gkdataset(4500000)
%timeit gk_df.to_csv("gktest.csv")
%timeit gk_df_csv = pd.read_csv("gktest.csv")

21.7 s ± 533 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
3.27 s ± 19.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [122]:

# Pickle File
gk_df = get_gkdataset(4500000)
%timeit gk_df.to_pickle("gktest.pickle")
%timeit gk_df_pickle = pd.read_pickle("gktest.pickle")

6.9 s ± 1.31 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
3.33 s ± 464 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [125]:
# Parquet File
gk_df = get_gkdataset(4500000)
%timeit gk_df.to_parquet("gktest.parquet")
%timeit gk_df_parquet = pd.read_parquet("gktest.parquet")

1.85 s ± 123 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
591 ms ± 119 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [128]:
# Feather File

gk_df = get_gkdataset(4500000)
%timeit gk_df.to_feather("gktest.feather")
%timeit gk_df_feather = pd.read_feather("gktest.feather")

864 ms ± 24.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
454 ms ± 13.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [129]:
!ls -GFlash gktest.csv gktest.pickle gktest.parquet gktest.feather

491528 -rw-r--r--  1 user  staff   238M Oct 15 15:25 gktest.csv
295208 -rw-r--r--  1 user  staff   130M Oct 15 15:40 gktest.feather
 98320 -rw-r--r--  1 user  staff    48M Oct 15 15:33 gktest.parquet
392664 -rw-r--r--  1 user  staff   192M Oct 15 15:30 gktest.pickle
